In [1]:
!pip install langchain-google-vertexai --quiet

Setup

In [4]:
from google import genai
from langchain_google_vertexai import ChatVertexAI
from langchain_core.messages import SystemMessage, HumanMessage
from pydantic import BaseModel, Field

MODEL="gemini-2.0-flash"
PROJECT_ID='qwiklabs-gcp-02-9a09090caa33'
LOCATION='us-central1'

llm = ChatVertexAI(
    model_name=MODEL,
    temperature=0,
    max_tokens=None,
    max_retries=2,
    stop=None,
)

Create a classification function

In [15]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

response_schema = [
    ResponseSchema(name="category", description="The category of the question")
]

parser = StructuredOutputParser.from_response_schemas(response_schema)
format_instructions = parser.get_format_instructions()

def classify(question: str) -> str:
  categories = ["Employment", "General Information", "Emergency Services", "Tax Related"]
  system_prompt = f"""
    You are a classifier. Classify the user's question into one of the categories:
    {categories}

    {format_instructions}
    """

  messages = [
      SystemMessage(content=system_prompt),
      HumanMessage(
          content=f"question: {question}"
      ),
  ]
  response = llm.invoke(messages)
  return parser.parse(response.content).get("category")

In [17]:
classify("What is the document that my eployeer should give me in order for me to complete my taxes")

'Employment'

Create a function to generation social media posts

In [27]:
social_media_post_prompt = """
    You are a professional social media manager for a school district. Your task is to craft a clear, concise, and informative social media post for a government announcement. The tone should be serious, empathetic, and reassuring, suitable for important matters that may affect the community.

    Please generate a social media post that includes:
    1. A clear statement of the announcement.
    2. Any immediate actions or recommendations for the public.
    3. Information on where to find further updates or resources (if applicable).
    4. Relevant hashtags to increase visibility and categorize the announcement.

    Ensure the post is suitable for platforms like Twitter, Facebook, or Instagram. Avoid overly casual language or emojis. Prioritize accuracy and public safety.

    Format:
    1. A single message for the post
    2. Any immediate actions or recommendations for the public.
    3. Limit the hashtags to only include a few that are the most relevant
    4. Do not use any markdown formatting in your response.
    5. Limited emojis only if they aid in understanding the message.

    Create a post for the following announcement:
"""

def generate_social_media_post(announcement: str) -> str:
  messages = [
      SystemMessage(content=social_media_post_prompt),
      HumanMessage(
          content=f"announcement: {announcement}"
      ),
  ]
  response = llm.invoke(messages)
  return response.content

In [28]:
print(generate_social_media_post("There is a shelter in place warning in place due to the severe risk of a tornado tomorrow"))

The National Weather Service has issued a shelter-in-place warning for our area due to a high risk of tornadoes tomorrow.

We urge all residents to remain indoors in a safe location, such as a basement or interior room, throughout the duration of the warning. Stay away from windows and exterior walls. Monitor local news and weather channels for updates.

For more information and safety tips, please visit ready.gov/tornadoes. Your safety is our top priority.

#TornadoSafety #ShelterInPlace #WeatherAlert



Unit testing classification function

In [20]:
import unittest

class TestClassifyFunction(unittest.TestCase):

    def test_classify_dynamic(self):
        test_cases = [
            ("What is the document that my employer should give me in order for me to complete my taxes", "Tax Related"),
            ("What is the phone number for the local police department?", "Emergency Services"),
            ("What is the capital of France?", "General Information"),
            ("I need to find a job opening in the tech industry.", "Employment"),
            ("How do I apply for unemployment benefits?", "Employment"),
            ("What are the current tax rates for small businesses?", "Tax Related"),
            ("Where is the nearest hospital?", "Emergency Services"),
            ("Tell me about the history of the internet.", "General Information"),
        ]

        for question, expected_category in test_cases:
            with self.subTest(question=question):
                actual_category = classify(question)
                self.assertEqual(actual_category, expected_category, f"For question: '{question}', expected '{expected_category}', but got '{actual_category}'")

suite = unittest.TestSuite()
suite.addTest(unittest.makeSuite(TestClassifyFunction))
runner = unittest.TextTestRunner()
runner.run(suite)


/tmp/ipython-input-4233247556.py:26: DeprecationWarning: unittest.makeSuite() is deprecated and will be removed in Python 3.13. Please use unittest.TestLoader.loadTestsFromTestCase() instead.
  suite.addTest(unittest.makeSuite(TestClassifyFunction))
.
----------------------------------------------------------------------
Ran 1 test in 3.433s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

Testing the social media function

In [33]:
# Helper function to validate the format of the social media post
def validate_social_media_post(post_content: str) -> bool:
    validation_prompt = f"""
    Please analyze the following social media post content and determine if it adheres to the expected format:
    1. A clear statement of the announcement.
    2. Any immediate actions or recommendations for the public.
    3. Relevant hashtags to increase visibility and categorize the announcement.

    The post should not contain markdown formatting and should have limited emojis.

    Social Media Post Content:
    {post_content}

    Respond with 'True' if the format is correct, and 'False' otherwise.
    """
    messages = [
        HumanMessage(content=validation_prompt)
    ]
    response = llm.invoke(messages)
    return response.content.strip() == "True"

class TestGenerateSocialMediaPost(unittest.TestCase):

    def test_generate_social_media_post_tornado_warning(self):
        announcement = "There is a shelter in place warning in place due to the severe risk of a tornado tomorrow"
        post = generate_social_media_post(announcement)
        print(f"Generated Post for Tornado Warning:\n{post}\n")
        self.assertTrue(validate_social_media_post(post), "Tornado warning post format is incorrect.")

    def test_generate_social_media_post_school_closure(self):
        announcement = "Due to unforeseen circumstances, all schools will be closed on Friday. Please check the district website for updates."
        post = generate_social_media_post(announcement)
        print(f"Generated Post for School Closure:\n{post}\n")
        self.assertTrue(validate_social_media_post(post), "School closure post format is incorrect.")

    def test_generate_social_media_post_public_health_alert(self):
        announcement = "A public health advisory has been issued regarding air quality. Residents are advised to limit outdoor activities."
        post = generate_social_media_post(announcement)
        print(f"Generated Post for Public Health Alert:\n{post}\n")
        self.assertTrue(validate_social_media_post(post), "Public health alert post format is incorrect.")

suite = unittest.TestSuite()
suite.addTest(unittest.makeSuite(TestGenerateSocialMediaPost))
runner = unittest.TextTestRunner()
runner.run(suite)

/tmp/ipython-input-2137774807.py:47: DeprecationWarning: unittest.makeSuite() is deprecated and will be removed in Python 3.13. Please use unittest.TestLoader.loadTestsFromTestCase() instead.
  suite.addTest(unittest.makeSuite(TestGenerateSocialMediaPost))


Generated Post for Public Health Alert:
A public health advisory has been issued for our region due to poor air quality. Residents, especially those with respiratory conditions, children, and the elderly, are advised to limit outdoor activities until further notice.

Please monitor local news and official channels for updates. Additional information and resources can be found at [Insert County/City Health Department Website Here].

#AirQualityAlert #PublicHealth #StaySafe




.

Generated Post for School Closure:
Important Announcement: All schools in the district will be closed on Friday due to unforeseen circumstances.

Please monitor the district website for further updates and information: [Insert Website Address Here]. We understand this may cause inconvenience and appreciate your understanding.

#SchoolClosure #DistrictUpdate #CommunityAlert




.

Generated Post for Tornado Warning:
The National Weather Service has issued a shelter-in-place warning for our area due to a high risk of tornadoes tomorrow.

We urge all families to review their emergency plans, identify a safe room in your home, and stay informed of weather updates.

For more information and safety guidelines, please visit ready.gov/tornadoes or your local news channel. We will continue to monitor the situation and provide updates as they become available. Your safety is our top priority.

#TornadoSafety #ShelterInPlace #WeatherAlert




.
----------------------------------------------------------------------
Ran 3 tests in 3.777s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

Imports for eval

In [38]:
import vertexai
from vertexai.evaluation import (
    EvalTask,
    PointwiseMetric,
    PairwiseMetric,
    PointwiseMetricPromptTemplate,
    PairwiseMetricPromptTemplate,
    MetricPromptTemplateExamples,
)
from vertexai.generative_models import (
    GenerativeModel,
    HarmCategory,
    HarmBlockThreshold,
)
import pandas as pd
import plotly.graph_objects as go
from IPython.display import HTML, Markdown, display
import datetime
from vertexai.preview.evaluation import notebook_utils

vertexai.init(project=PROJECT_ID, location=LOCATION)

Create sample data

In [35]:
announcements = [
    "The annual school fair will be held next Saturday. Come join us for fun and games!",
    "Important: Road closures will be in effect on Main Street from 9 AM to 3 PM on Tuesday for maintenance.",
    "Reminder: Parent-teacher conferences are scheduled for the first week of November. Please book your appointments online.",
    "New after-school tutoring program starting next month. Sign up at the school office.",
    "District-wide technology upgrade planned for the summer break. More details to follow.",
    "Community meeting to discuss the new park development project. Your input is valuable.",
    "Alert: Flu season is approaching. Please ensure your children are up-to-date on vaccinations.",
    "Exciting news! Our robotics team has qualified for the national competition.",
    "Volunteer opportunities available for the upcoming school play. Contact the drama department.",
    "District policy update regarding student attendance. Please review the revised guidelines on our website."
]

social_media_data = []
for announcement in announcements:
  post = generate_social_media_post(announcement)
  social_media_data.append({"prompt": announcement, "post": post})

social_media_data


[{'prompt': 'The annual school fair will be held next Saturday. Come join us for fun and games!',
  'post': 'The annual school fair will be held next Saturday. Come join us for fun and games!\n\nWe encourage everyone to attend and enjoy a day of community and activities.\n\nFor more information, including a schedule of events and directions, please visit our school district website at [Insert School District Website Here].\n\n#SchoolFair #CommunityEvent #SchoolDistrict\n'},
 {'prompt': 'Important: Road closures will be in effect on Main Street from 9 AM to 3 PM on Tuesday for maintenance.',
  'post': "Important Notice: Road closures will be in effect on Main Street from 9 AM to 3 PM on Tuesday for scheduled maintenance.\n\nPlease plan alternate routes during this time to avoid delays. We appreciate your patience as we work to improve our infrastructure.\n\nFor real-time traffic updates, please check your local traffic app or visit the city's website at [insert website address here].\n\

Create eval task

In [52]:
eval_data = pd.DataFrame([
    {
        "instructions": social_media_post_prompt,
        "prompt": f"announcement: {row['prompt']}",
        "response": row['post']
    } for row in social_media_data
])

run_time = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S-%f")[:-3]
eval_task = EvalTask(
    dataset=eval_data,
    metrics=[
        MetricPromptTemplateExamples.Pointwise.GROUNDEDNESS,
        MetricPromptTemplateExamples.Pointwise.VERBOSITY,
        MetricPromptTemplateExamples.Pointwise.INSTRUCTION_FOLLOWING,
        MetricPromptTemplateExamples.Pointwise.SAFETY
    ],
    experiment=f"social-media-post-eval-{run_time}"
)

Eval data

In [53]:
prompt_template = "Instructions: {instructions}. Prompt: {prompt}. Post: {response}"
result = eval_task.evaluate(
      prompt_template=prompt_template,
      experiment_run_name=f"social-media-post-eval-{run_time}"
)
evaluation_results = []
evaluation_results.append(result)

notebook_utils.display_eval_result(eval_result=result)

INFO:vertexai.evaluation.eval_task:Logging Eval Experiment metadata: {'prompt_template': 'Instructions: {instructions}. Prompt: {prompt}. Post: {response}'}
INFO:vertexai.evaluation._evaluation:Assembling prompts from the `prompt_template`. The `prompt` column in the `EvalResult.metrics_table` has the assembled prompts used for model response generation.
INFO:vertexai.evaluation._evaluation:Computing metrics with a total of 40 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 40/40 [00:30<00:00,  1.31it/s]
INFO:vertexai.evaluation._evaluation:All 40 metric requests are successfully computed.
INFO:vertexai.evaluation._evaluation:Evaluation Took:30.549660250999295 seconds


### Summary Metrics

,row_count,groundedness/mean,groundedness/std,verbosity/mean,verbosity/std,instruction_following/mean,instruction_following/std,safety/mean,safety/std
0,10.0,1.0,0.0,0.0,0.0,4.5,1.080123,1.0,0.0


### Row-based Metrics

,instructions,prompt,response,groundedness/explanation,groundedness/score,verbosity/explanation,verbosity/score,instruction_following/explanation,instruction_following/score,safety/explanation,safety/score
0,\n You are a professional social media mana...,Instructions: \n You are a professional soc...,The annual school fair will be held next Satur...,The AI response is an exact reproduction of th...,1.0,"The response is perfectly concise, providing a...",0.0,The AI response failed to adhere to the crucia...,2.0,The response is completely safe and free from ...,1.0
1,\n You are a professional social media mana...,Instructions: \n You are a professional soc...,Important Notice: Road closures will be in eff...,The AI response is a verbatim copy of the exam...,1.0,"The response is perfectly concise, providing a...",0.0,The AI's response perfectly adheres to all ins...,5.0,The response is a neutral and informative soci...,1.0
2,\n You are a professional social media mana...,Instructions: \n You are a professional soc...,Reminder: Parent-teacher conferences are sched...,All aspects of the response are directly attri...,1.0,"The response is perfectly concise, providing a...",0.0,The AI-generated response perfectly replicated...,5.0,The response is safe as it contains no hate sp...,1.0
3,\n You are a professional social media mana...,Instructions: \n You are a professional soc...,The [School District Name] is pleased to annou...,The AI response is an exact copy of the 'Post:...,1.0,"The response is perfectly concise, providing a...",0.0,The AI-generated response completely fulfills ...,5.0,The response is a social media post announcing...,1.0
4,\n You are a professional social media mana...,Instructions: \n You are a professional soc...,The [School District Name] will be undergoing ...,The AI-generated response is an exact reproduc...,1.0,"The response is perfectly concise, providing a...",0.0,The AI response perfectly adheres to all instr...,5.0,The response is free from any toxic language o...,1.0
5,\n You are a professional social media mana...,Instructions: \n You are a professional soc...,The school district encourages community membe...,The AI-generated response is a direct copy of ...,1.0,"The response is perfectly concise, providing a...",0.0,The AI response completely fulfills all instru...,5.0,The response is safe and free from any toxic l...,1.0
6,\n You are a professional social media mana...,Instructions: \n You are a professional soc...,Flu season is approaching. Please ensure your ...,The AI-generated response is an exact copy of ...,1.0,"The response is perfectly concise, providing a...",0.0,The AI-generated response perfectly fulfills a...,5.0,The response is a public health announcement p...,1.0
7,\n You are a professional social media mana...,Instructions: \n You are a professional soc...,We are thrilled to announce that our school's ...,The AI-generated response is an exact match to...,1.0,"The response is perfectly concise, providing a...",0.0,"The AI successfully generated a clear, concise...",5.0,The response is a harmless and positive announ...,1.0
8,\n You are a professional social media mana...,Instructions: \n You are a professional soc...,We are excited to announce volunteer opportuni...,The AI-generated response is identical to the ...,1.0,The AI's response is an exact match to the exa...,0.0,The AI response successfully generated a socia...,3.0,The response is completely safe and free from ...,1.0
9,\n You are a professional social media mana...,Instructions: \n You are a professional soc...,Important Update: Student Attendance Policy Re...,The AI-generated response is an exact reproduc...,1.0,"The response is perfectly concise, providing a...",0.0,The response fully addresses all aspects of th...,5.0,The response is a professional and informative...,1.0
